<a href="https://colab.research.google.com/github/fatenchtioui/NLP/blob/main/nlp_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def traiter(url):
  # initilise deux listes vides pour stocker les pharses et les classes
  pharses=[]
  classes=[]
  #ouvre le fichier en utilisent l'URL fournie en mode lecture ('r)et en spécifient l'encodage UTF-8
  with open (url, mode='r', encoding='utf-8')as f:
  #parcours chaque ligne dans le fiche
      for ligne in f:
      #deivise la ligne en utiliseant le caractére de tabulation '\t' comme délimiteur
         parties = ligne.split('\t')
      #ajoute la quatriéme partir (indice 3) à la liste des pharase
         pharses.append(parties[3])
# convertit la deuxiéme partie (indice1) en entier et l'ajoute à la liste des classes
         classes.append(int(parties[1]))
# returne les listes des pharases et de classes
  return  pharses, classes

In [ ]:
X_train, Y_train = traiter('/content/drive/MyDrive/folder/in_domain_train.tsv')
X_train[:3], Y_train[:3]

(["our friends wo n't buy this analysis , let alone the next one we propose .\n",
  "one more pseudo generalization and i 'm giving up .\n",
  "one more pseudo generalization or i 'm giving up .\n"],
 [1, 1, 1])

In [ ]:
X_dev, Y_dev = traiter('/content/drive/MyDrive/folder/in_domain_dev.tsv')
X_dev[:3], Y_dev[:3]

(['the sailors rode the breeze clear of the rocks .\n',
  'the weights made the rope stretch over the pulley .\n',
  'the mechanical doll wriggled itself loose .\n'],
 [1, 1, 1])

## création et utilisation d'un modéle de langage N_gramme basé sur la bibliothéqie NLTK

In [ ]:
from nltk.lm import MLE, Laplace, KneserNeyInterpolated
from nltk.lm import Vocabulary
from nltk.lm.preprocessing import pad_both_ends,padded_everygram_pipeline
class NGramme:
    #n= 2 la taille des N-grammes
    #cutoff=2 --> tous les mots qui apparaissent moins de 2 fois dans les données
    # d'apprentissage seront traités comme des mots rares ou inconnus
    def __init__(self, n=2, cutoff=2):
        self.n = n
        # créer un modèle de langage Kneser-Ney Interpolated qui utilise des N-grammes
        #de taille n et un vocabulaire qui gère les mots rares ou inconnus en fonction  de cutoff=2.
        # Le modèle sera utilisé pour estimer les probabilités des séquences de mots dans le texte,
        # et le vocabulaire configuré aidera à gérer les mots peu fréquents de manière efficace.
        self.kn = KneserNeyInterpolated(n, vocabulary=Vocabulary(unk_cutoff=cutoff))

    def preprocess(self, X):
        # liste pour stocker les phrases prétraitées
        sent_words = []
        #pour chaque phrase
        for sent in X:
            # Divise chaque phrase en mots et ajoute un padding au début et à la fin pour les n-grams
            sent_words.append(list(pad_both_ends(sent.split(), self.n)))
            #sent.split(): diviser la phrase en mots (\t)
            #pad_both_ends(): ajouter un padding <s> </s> au début et à la fin de la liste de mots.
        return sent_words

    #Fonct pour entraîner le modèle de langage Kneser-Ney Interpolated sur les données textuelles prétraitées
    def entrainer(self, X):
        sent_words = self.preprocess(X)
        #train = les N-grammes extraits des données d'apprentissage
        #vocab = un objet de vocabulaire qui stocke les N-grammesuniques extraits du corpus et
        #aide le modèle de langage à effectuer des calculs sur les données d'apprentissage
        train, vocab = padded_everygram_pipeline(self.n, sent_words)
        vocab_l = list(vocab)
        train_l = []
        for t in train:
            # prépare les données pour l'entraînement du modèle ( transformer chaque n-gramm en liste)
            train_l.append(list(t))
        # Entraîne le modèle Kneser-Ney Interpolated sur les données
        #Le modèle sera ajusté pour estimer les probabilités des N-grammes en fonction des données d'apprentissage.
        self.kn.fit(train_l, vocab_l)




In [ ]:
def noter_pharse(self, phrase):
  notes=[]# liste pour stocker les notes
  print('notation de la phrase:',phrase)
  # parcourir la phrase en considérant des sous-séquence de mots de la taille des N-gramme
  for i in range(len(phrase)-self.n + 1):
    notes.append(self.kn.score(phrase(i+self.n-1), phrase[i:i + self.n- 1]))
    return notes
    # scores pour tout le corpus
    def noter(self, X):
      notes = []
      sent_Word = self.preprocess(X)
      for phrase in sent_Word:
        notes.append(self.noter_pharse(phrase))
        return notes
        ngram = NGramme(3)
        ngram.entrainer(X_train)
        ngram.noter(X_train[:3])


In [ ]:
from pickle import LONG_BINGET
from nltk.util import ngrams
import random
def generer_texte(modele, mopt_depart, longueur=5):
  texte_genere=[mopt_depart]
  for _ in range(longueur):
    #les probabilité des mots suivante en utilisant le modéle
    prob_max =-float("inf")
    # initialisation la variantes prob_max pour une valeur négative infinie
    mot_suivant=""
    phrase = texte_genere[-(modele.n-1):]
    #initialise la varable phrase en extrayant le N-1
    for mot in modele.kn.vocab:
      print("-------------------")
      print(mot)
      phrase.append(mot)
      prob = modele.kn.score(mot, phrase[:1])
      print("----------------")
      print(prob)
      if prob > prob_max and mot != "</s>":
        prob_max = prob
        mot_suivant = mot
        texte_genere.append(mot_suivant)
        if  mot_suivant == "</s>":
          break
          texte_genere = "" .join(texte_genere)
          return texte_genere
          mopt_depart = "our"
          texte_genere = generer_texte(ngram, mopt_depart, longueur=3)
          print(texte_genere)

